This notebook compares the overfitting of Fairlearn Vs OxonFair using random forests and decision trees on the adult dataset.

We use sex as the protected attribute.

Even on this low-dimensional data, the default parameters of scikit-learn cause both decision trees and random forests to overfit. 

The models obtain 0 error on the training set. As a consequence of this, defintions such as equal opportunity are trivially satisfied, and fairness methods such as fairlearn which enforce fairness on the training set do not work.

This overfitting, and the consequential failure of fairness methods to work can be avoided by specifying a low maximimal tree depth. The examples in Fairlearn documentation typically use a tree depth of 4 on adult. 

Oxonfair allows for the enforcing of fairness on validation data, and this means that it can enforce fairness even when the training error is zero. 

In [1]:
from oxonfair import FairPredictor, performance, dataset_loader
from oxonfair import group_metrics as gm
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train,val,test = dataset_loader.adult()
basetree = DecisionTreeClassifier().fit(X=train['data'], y=train['target'])
baseforest = RandomForestClassifier().fit(X=train['data'], y=train['target'])

We now specify a fair predictors over the validation set.

In [3]:
# The outputs of a decision tree are all 0 or 1, so we add Gaussian noise to allow thresholding to work
ftree=FairPredictor(basetree,val,add_noise=0.001)
fforest=FairPredictor(baseforest,val)

We call fit to enforce equal opportunity.

In [4]:
ftree.fit(gm.accuracy,gm.equal_opportunity,0.02)
fforest.fit(gm.accuracy,gm.equal_opportunity,0.02)

We now focus on trees only.
And evaluate fairness on validation data.

In [5]:
ftree.evaluate_fairness()

,original,updated
Statistical Parity,0.189422,0.162837
Predictive Parity,0.092910,0.092894
Equal Opportunity,0.053580,0.000976
Average Group Difference in False Negative Rate,0.053580,0.000976
Equalized Odds,0.080858,0.047254
Conditional Use Accuracy,0.102704,0.111073
Average Group Difference in Accuracy,0.130122,0.136368
Treatment Equality,0.178016,0.382540


And on the test set.

In [6]:
ftree.evaluate_fairness(test)

,original,updated
Statistical Parity,0.188561,0.163937
Predictive Parity,0.103807,0.104589
Equal Opportunity,0.072807,0.023210
Average Group Difference in False Negative Rate,0.072807,0.023210
Equalized Odds,0.087895,0.056234
Conditional Use Accuracy,0.106547,0.114582
Average Group Difference in Accuracy,0.124948,0.130461
Treatment Equality,0.133225,0.323895


We now check validation performance.

In [7]:
ftree.evaluate()

,original,updated
Accuracy,0.807781,0.804177
Balanced Accuracy,0.741476,0.724565
F1 score,0.604682,0.582941
MCC,0.477844,0.455199
Precision,0.595357,0.594450
Recall,0.614305,0.571869
ROC AUC,0.741476,0.707625


And on the test set.

In [8]:
ftree.evaluate(test)

,original,updated
Accuracy,0.811236,0.808779
Balanced Accuracy,0.742806,0.727586
F1 score,0.607927,0.588691
MCC,0.483642,0.464606
Precision,0.604329,0.606534
Recall,0.611567,0.571869
ROC AUC,0.742806,0.710538


We now run fairlearn on the same data.

In [9]:
from fairlearn.reductions import TruePositiveRateParity, ExponentiatedGradient
mitagator = ExponentiatedGradient(DecisionTreeClassifier(),TruePositiveRateParity())
mitagator.fit(X=train['data'],y=train['target'],sensitive_features=train['data']['sex'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x16dc12710>,
                      estimator=DecisionTreeClassifier(),
                      nu=2.0474182056426843e-05)

To evaluate fairlearn, we write a helper function to evaluate performance and fairness on train or test, and concat the outputs together.  

In [10]:
def eval(train, classifier=mitagator):
    return pd.concat((performance.evaluate(train['target'], classifier.predict(train['data'])),
                      performance.evaluate_fairness(train['target'], classifier.predict(train['data']), train['groups'])),axis=0)

out = pd.concat((eval(train), eval(test)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999959,0.811563
Balanced Accuracy,0.999973,0.742435
F1 score,0.999914,0.607673
MCC,0.999888,0.483684
Precision,0.999829,0.605505
Recall,1.000000,0.609856
ROC AUC,0.999973,0.742435
Statistical Parity,0.194639,0.182781
Predictive Parity,0.000202,0.109963
Equal Opportunity,0.000000,0.060128


Evaluating the initially trained baseline classifier we find that, as expected, fairlearn did not substantially alter the performance or unfairness of the classifier (beyond altering the random seed of the tree).

In [11]:
out = pd.concat((eval(train, basetree), eval(test, basetree)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999959,0.811236
Balanced Accuracy,0.999914,0.742806
F1 score,0.999914,0.607927
MCC,0.999888,0.483642
Precision,1.000000,0.604329
Recall,0.999829,0.611567
ROC AUC,0.999914,0.742806
Statistical Parity,0.194516,0.188561
Predictive Parity,0.000000,0.103807
Equal Opportunity,0.000202,0.072807


We now do the same with the random forest classifier.

In [12]:
fforest.evaluate_fairness()

,original,updated
Statistical Parity,0.170142,0.138165
Predictive Parity,0.006499,0.031893
Equal Opportunity,0.062601,0.012177
Average Group Difference in False Negative Rate,0.062601,0.012177
Equalized Odds,0.066470,0.029292
Conditional Use Accuracy,0.055173,0.074444
Average Group Difference in Accuracy,0.113257,0.111909
Treatment Equality,0.138755,0.049118


In [13]:
fforest.evaluate_fairness(test)

,original,updated
Statistical Parity,0.180806,0.146137
Predictive Parity,0.005442,0.041881
Equal Opportunity,0.114180,0.056922
Average Group Difference in False Negative Rate,0.114180,0.056922
Equalized Odds,0.094411,0.053377
Conditional Use Accuracy,0.051040,0.076988
Average Group Difference in Accuracy,0.108786,0.108908
Treatment Equality,0.209692,0.001891


In [14]:
fforest.evaluate()

,original,updated
Accuracy,0.854627,0.855528
Balanced Accuracy,0.767929,0.753861
F1 score,0.664525,0.649304
MCC,0.578162,0.573177
Precision,0.742085,0.774668
Recall,0.601643,0.558864
ROC AUC,0.903254,0.895934


In [15]:
fforest.evaluate(test)

,original,updated
Accuracy,0.852919,0.852838
Balanced Accuracy,0.767506,0.750798
F1 score,0.662660,0.643523
MCC,0.574235,0.565098
Precision,0.734388,0.765455
Recall,0.603696,0.555099
ROC AUC,0.904493,0.897846


In [16]:
mitagator = ExponentiatedGradient(RandomForestClassifier(),TruePositiveRateParity())
mitagator.fit(X=train['data'],y=train['target'],sensitive_features=train['data']['sex'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x16de40220>,
                      estimator=RandomForestClassifier(),
                      nu=2.8954273085152237e-05)

In [17]:
out = pd.concat((eval(train,mitagator), eval(test,mitagator)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999959,0.854803
Balanced Accuracy,0.999914,0.769331
F1 score,0.999914,0.666165
MCC,0.999888,0.579339
Precision,1.000000,0.740477
Recall,0.999829,0.605407
ROC AUC,0.999914,0.769331
Statistical Parity,0.194516,0.180692
Predictive Parity,0.000000,0.005158
Equal Opportunity,0.000202,0.118862
